In [3]:
import numpy as np
import scipy as sp

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit.quantum_info import SparsePauliOp

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.settings import QiskitNatureSettings

QiskitNatureSettings.use_pauli_sum_op = False

# Generate parametrized quantum circuits with Qiskit

In [4]:
def get_hubbard_hamiltian(num_sites, t, U):
    """Constructs the Hubbard Hamiltonian in the dict format supported by qiskit-nature.
    Returns a list with two elements, the first being a dict containing the hopping terms,
    the second the onsite terms.
    """
    hopping = {}
    onsite = {}
    for n in range(num_sites - 1):
        hopping[f"+_{n + 1} -_{n}"] = -t
        hopping[f"+_{n} -_{n + 1}"] = -t
        hopping[f"+_{n + 1 + num_sites} -_{n + num_sites}"] = -t
        hopping[f"+_{n + num_sites} -_{n + 1 + num_sites}"] = -t

        onsite[f"+_{n} -_{n} +_{n + num_sites} -_{n + num_sites}"] = U
        onsite[f"+_{n + 1} -_{n + 1} +_{n + 1 + num_sites} -_{n + 1 + num_sites}"] = U

    return [hopping, onsite]

In [5]:
# Define the system parameters
num_sites = 2
t = 1
U = 1

# get the dictionary of second quantization operators
hubbard_ops = get_hubbard_hamiltian(num_sites, t, U)

# map the second quantization operators to Pauli operators
num_orbitals = 2 * num_sites
ops_mapped = []
for term in hubbard_ops:
    op = FermionicOp(
        term,
        num_spin_orbitals=num_orbitals,
    )
    print(op)

    ops_mapped.append(JordanWignerMapper().map(op))

# get the Hamiltonian matrix
H = sum(ops_mapped)

Fermionic Operator
number spin orbitals=4, number terms=4
  -1 * ( +_1 -_0 )
+ -1 * ( +_0 -_1 )
+ -1 * ( +_3 -_2 )
+ -1 * ( +_2 -_3 )
Fermionic Operator
number spin orbitals=4, number terms=2
  1 * ( +_0 -_0 +_2 -_2 )
+ 1 * ( +_1 -_1 +_3 -_3 )


In [6]:
init_circ = QuantumCircuit(num_orbitals)
for i in range(int(num_sites / 2)):
    init_circ.x([i, i + num_sites])

ansatz = EvolvedOperatorAnsatz(
    operators=[SparsePauliOp(op.paulis) for op in ops_mapped],
    reps=2,
    initial_state=init_circ,
    insert_barriers=True,
)

In [7]:
print(ansatz.decompose().decompose().decompose())

global phase: -1.0*t[1] - 1.0*t[3]
     ┌──────────┐┌────────────────┐┌────────────────┐┌──────────────┐»
q_0: ┤ U(π,0,π) ├┤0               ├┤0               ├┤ Rz(2.0*t[1]) ├»
     └──────────┘│  Rxx(2.0*t[0]) ││  Ryy(2.0*t[0]) │├──────────────┤»
q_1: ────────────┤1               ├┤1               ├┤ Rz(2.0*t[1]) ├»
     ┌──────────┐├────────────────┤├────────────────┤├──────────────┤»
q_2: ┤ U(π,0,π) ├┤0               ├┤0               ├┤ Rz(2.0*t[1]) ├»
     └──────────┘│  Rxx(2.0*t[0]) ││  Ryy(2.0*t[0]) │├──────────────┤»
q_3: ────────────┤1               ├┤1               ├┤ Rz(2.0*t[1]) ├»
                 └────────────────┘└────────────────┘└──────────────┘»
«                                    ░ ┌────────────────┐┌────────────────┐»
«q_0: ─■─────────────────────────────░─┤0               ├┤0               ├»
«      │                             ░ │  Rxx(2.0*t[2]) ││  Ryy(2.0*t[2]) │»
«q_1: ─┼──────────────■──────────────░─┤1               ├┤1               ├»
«      │ZZ(2.0*t[1

In [13]:
param_vals = np.random.rand(ansatz.num_parameters)

bound_circ = ansatz.bind_parameters(
    {x_i: param_vals[i] for i, x_i in enumerate(ansatz.parameters)}
)

In [20]:
print(ansatz)
print(bound_circ)
print(bound_circ.decompose().decompose().decompose())

     ┌──────────────────────────────────┐
q_0: ┤0                                 ├
     │                                  │
q_1: ┤1                                 ├
     │  EvolvedOps(t[0],t[1],t[2],t[3]) │
q_2: ┤2                                 ├
     │                                  │
q_3: ┤3                                 ├
     └──────────────────────────────────┘
     ┌──────────────────────────────────────────────┐
q_0: ┤0                                             ├
     │                                              │
q_1: ┤1                                             ├
     │  EvolvedOps(0.60519,0.44408,0.55243,0.73077) │
q_2: ┤2                                             ├
     │                                              │
q_3: ┤3                                             ├
     └──────────────────────────────────────────────┘
global phase: 5.1083
     ┌──────────┐┌──────────────┐┌──────────────┐┌─────────────┐              »
q_0: ┤ U(π,0,π) ├┤0             ├┤0

# Cost function evaluation using SciPy

In [8]:
def cost_fun(x, coeff):
    cost = coeff[0] * (x[0] - 1) ** 2 + coeff[1] * (x[1] - 2.5) ** 2
    print(cost)
    return cost

In [10]:
coeff = [2, 3]
init_vals = [2, 0]

opt_res = sp.optimize.minimize(cost_fun, init_vals, args=(coeff), method="COBYLA", options={"maxiter": 1000})
print(opt_res)

20.75
26.75
8.75
1.711300899000926
2.650751504576093
0.26710182256377607
3.5826240683041894
0.3952800854608933
0.1818218389389144
0.40572580006566655
0.005767871210886576
0.16358060374081046
0.015185385551978962
0.017678215951285724
0.019555462304911973
0.0010243687123151907
0.0037065460937771204
0.0003577053117217971
0.0019212175319351038
0.0005069289372852195
0.00010731261230146431
0.0001754921190776399
1.8489151149912667e-05
0.00011024678719806189
3.410640002194436e-05
5.821045493525303e-06
2.1517235291323945e-05
1.6636113595986393e-07
3.883023084480454e-06
2.6810526529008448e-06
1.1900201989216171e-07
4.1518742585949045e-07
2.0415546553708333e-07
1.1065891791714441e-07
1.6201520847598855e-07
4.002888559864938e-08
1.0634020531221324e-08
6.230582103240182e-08
4.252795186348859e-09
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 4.252795186348859e-09
       x: [ 1.000e+00  2.500e+00]
    nfev: 39
   maxcv: 0.0
